In [2]:
import requests
import json
import os
from dotenv import load_dotenv
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import MarianMTModel, MarianTokenizer
import faiss

/Users/brandonamarasingam/anaconda3/envs/pytorch/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/brandonamarasingam/anaconda3/envs/pytorch/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/brandonamarasingam/anaconda3/envs/pytorch/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: (__ZN3c1017RegisterOperatorsD1Ev)
  Referenced from: '/Users/brandonamarasingam/anaconda3/envs/pytorch/lib/python3.11/site-packages/torchvision/image.so'
  Expected in: '/Users/brandonamarasingam/anaconda3/envs/pytorch/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Other

In [3]:
# Load environment variables
load_dotenv()

# Groq API setup
GROQ_API = os.getenv("GROQ_API")

In [ ]:
# Function to get response from Groq API
def get_groq_response(prompt):
    url = "https://api.groq.com/v1/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {GROQ_API}"
    }
    data = {
        "model": "groq-model-name",  # Replace with the actual Groq model name
        "prompt": prompt,
        "max_tokens": 100
    }
    
    response = requests.post(url, headers=headers, data=json.dumps(data))
    if response.status_code == 200:
        return response.json()['choices'][0]['text'].strip()
    else:
        return f"Error: {response.status_code} - {response.text}"
